In [7]:
import pandas as pd
from collections import Counter
from cleanco import cleanco
import re as regex
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')
# nltk.download('popular')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Manish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\Manish\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\Manish\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\Manish\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\Manish\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\User

True

In [8]:
def sequence_uniqueness(seq, token2frequency):
    return sum(1/token2frequency[t]**0.5 for t in seq)

def name_similarity(a, b, token2frequency):
    a_tokens = set(a)
    b_tokens = set(b)
    a_uniq = sequence_uniqueness(a, token2frequency)
    b_uniq = sequence_uniqueness(b, token2frequency)
    if a_uniq==0 or b_uniq == 0:
        return 0
    else:
        return sequence_uniqueness(a_tokens.intersection(b_tokens), token2frequency)/(a_uniq * b_uniq) ** 0.5

def parse_name(name):
    name = cleanco(name).clean_name()
    #name = name.translate(None, string.punctuation)
    name = regex.sub(r"[[:punct:]]+", "", name)
    tokens = nltk.word_tokenize(name) 
    tokens = [t.lower() for t in tokens]
    tokens = [t for t in tokens if t not in stopwords.words('english')] 
    return tokens

def build_token2frequency(names):
    alltokens = []
    for tokens in names.values():
        alltokens += tokens

    return Counter(alltokens)

In [16]:
def cleansing_special(name):
    dirts = [' ',';',':','.',',','*','#','@','|','\\','-','_','?','%','!','^','(',')','$','=','+','"','<','>',"'",]
    default_replace = dirts[0]
    
    for dirt in dirts[1:]:
        name = name.replace(dirt, default_replace)
    
    regex.sub(' +', ' ', name)
    temp_list = [i.strip() for i in name.split(default_replace)]
    temp_list = [i for i in temp_list if i]
    return " ".join(temp_list)

In [30]:
# with open('marchants.json') as merchantfile:
merchants = pd.read_excel('2000_Suppliers_August-20.xlsx')

merchants['Supp_Name1'] = merchants['Supp_Name'].apply(cleansing_special)


# merchants = merchants.Supp_Name.unique()

parsed_names = {merchant:parse_name(merchant) for merchant in merchants['Supp_Name1']}
token2frequency = build_token2frequency(parsed_names)

grouping = {}
for merchant, tokens in parsed_names.items():
    grouping[merchant] = {merchant2: name_similarity(tokens, tokens2, token2frequency) for merchant2, tokens2 in parsed_names.items()}

filtered_matches = {}
for merchant in grouping:
    filtered_matches[merchant] = {merchant1: ratio for merchant1, ratio in grouping[merchant].items() if ratio ==1 }

In [31]:
df = pd.concat({k: pd.DataFrame.from_dict(v, 'index') for k, v in filtered_matches.items()}, axis=0).reset_index()

In [32]:
# df = pd.concat({k: pd.DataFrame.from_dict(v, 'index') for k, v in grouping.items()}, axis=0).reset_index()
df.to_excel('2000_Supplier_Matched.xlsx')

In [27]:
grouping

{'SHANDONG IRON AND STEEL COMPANY LTD': {'SHANDONG IRON AND STEEL COMPANY LTD': 1.0,
  'UNIPARTS INDIA GMBH': 0.0,
  'SISECAM DIS TICARET A S': 0.0,
  'PANDITA INTERNATIONAL PTE LTD': 0.0,
  'VENCOREX THAILAND COMPANY LIMITED': 0.0,
  'SHIJIAZHUANG JACKCHEM CO LTD': 0.0,
  'MARS PACKING & WAREHOUSING INC': 0.0,
  'NILETTA MARINE INC': 0.0,
  'KELLOGGS COMPANY': 0.0,
  'HUNAN HUACHENG BIOTECH INC': 0.0,
  'SINOLITE INDUSTRIAL CO LIMITED': 0.0,
  'TOPHARMAN SHANGHAI CO LTD': 0.0,
  'DONGYING TIANDONG PHARMACEUTICAL CO LTD': 0.0,
  'CUBIC PHARMACEUTICALS LTD': 0.0,
  'TAEIK CO LTD': 0.0,
  'NOVOZYMES BERLIN GMBH': 0.0,
  'RENOLIT NEDERLAND BV': 0.0,
  'INDUCTOTHERM CORPORATION': 0.0,
  'M/S DECOR WALTHER ELNRICHTUNGS GMB': 0.0,
  'JLG EMEA BV': 0.0,
  'MARK ANDY PRINT PRODUCTS': 0.0,
  'PAILON INTERNATIONAL TRADING LIMITED': 0.0,
  'LOXY SHANGHAI CO LTD': 0.0,
  'ADI S R L': 0.0,
  '3M DEUTSCHLAND GMBH': 0.0,
  'ITALTILE CERAMICS PTY LTD': 0.0,
  'ATI FRP LOUISVILLE OPERATIONS': 0.0,
  'L

In [ ]:
# def cleansing_special(name):
name = 'SISECAM DIS TICARET A.S.  #$.    '
dirts = [' ',';',':','.',',','*','#','@','|','\\','-','_','?','%','!','^','(',')','$','=','+','"','<','>',"'",]
default_replace = dirts[0]

for dirt in dirts[1:]:
    name = name.replace(dirt, default_replace)

re.sub(' +', ' ', name)

In [ ]:
[i.strip() for i in name.split(default_replace)]

In [ ]:
[i for i in temp_list if i]

In [ ]:
print(" ".join(temp_list))